In [83]:
dataset_path =  "aniket-curlscape/pii-masking-english-100"

In [84]:
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset(dataset_path)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [85]:
dataset['train']

Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 100
})

In [86]:
import requests
import json
import time
from typing import Dict, Any, Optional

def call_chat_endpoint(content: str, model: str = "NousResearch/Meta-Llama-3-8B-Instruct") -> Optional[Dict[str, Any]]:
    """
    Call the chat completions endpoint with the given content.
    
    Args:
        content: The content to send as user message
        model: The model to use for completion
        
    Returns:
        Response dictionary or None if request failed
    """
    url = "http://localhost:8000/v1/chat/completions"
    
    payload = {
        "messages": [
            {
                "content": "You are a helpful assistant who can mask the PII data from text. Replace username with [USERNAME], email with [EMAIL], phone number with [PHONE], and address with [ADDRESS], and time with [TIME]",
                "role": "system"
            },
            {
                "content": content,
                "role": "user"
            }
        ],
        "model": model,
        "temperature": 1.0
    }
    
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=30)
        response.raise_for_status()
        return response.json().get('choices')[0].get('message').get('content')
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        return None

In [87]:
# Initialize list to store all responses
all_responses = []

# Iterate through the dataset
for i, item in enumerate(dataset['train']):
    # Access individual fields
    # id = item['id']
    question = item['source_text']
    result = item['target_text']
    
    # Record start time for processing
    start_time = time.time()
    llm_answer = call_chat_endpoint(question)
    print('-'*5)
    print(str({'Question': {question},
    'Actual answer': result,
    'LLM Answer': llm_answer}))
    print('-'*5)
    print('\n\n')
    end_time = time.time()
    processing_time = end_time - start_time

    if i > 5:
        break
    
    # # Create response object
    # response_data = {
    #     "item_index": i,
    #     "id": id,
    #     "question": question,
    #     "result": result,
    #     "llm_answer": llm_answer,
    #     "timestamp": time.time(),
    #     "processing_time_seconds": processing_time
    # }
    
    # # Add to list
    # all_responses.append(response_data)
    
    # # Save all responses to the same JSON file after each item
    # filename = "responses_pii_redaction.json"
    # try:
    #     with open(filename, 'w') as f:
    #         json.dump(all_responses, f, indent=2)
    #     print(f"Saved response for item {i} to {filename} (processed in {processing_time:.2f}s)")
    # except Exception as e:
    #     print(f"Error saving item {i}: {e}")

-----
{'Question': {'Subject: Group Messaging for Admissions Process\n\nGood morning, everyone,\n\nI hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:\n\n- wynqvrh053 - Meeting at 10:20am\n- luka.burg - Meeting at 21\n- qahil.wittauer - Meeting at quarter past 13\n- gholamhossein.ruschke - Meeting at 9:47 PM\n- pdmjrsyoz1460 '}, 'Actual answer': 'Subject: Group Messaging for Admissions Process\n\nGood morning, everyone,\n\nI hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:\n\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] ', 'LLM Answer': 'Subject: Group Messaging for Admiss

In [ ]:
"By using our online banking services, you agree to the following terms. Your account number is S17175131813, and your credit card number for any linked accounts is 4869-1774-7870-. Your date of birth is 1995-11-23, and your social security number is 253-18-4537. By using our online banking services, you agree to the following terms. Your account number is S17175131813, and your credit card number for any linked accounts is 4869-1774-7870-. Your date of birth is 1995-11-23, and your social security number is 253-18-4537. Please ensure that all information provided is accurate and up-to-date to facilitate"



In [1]:
# Check the baseline accuracy

In [2]:
import json
with open('responses.json', 'r') as f:
    responses = json.load(f)



In [11]:
responses[0]

{'item_index': 0,
 'id': 'math_qa__nwoNskhwavrdQAqt',
 'question': "The banker's gain of a certain sum due 3 years hence at 10% per annum is Rs. 36. What is the present worth ? Pick one:\nA) rs . 400  B) rs . 300  C) rs . 500  D) rs . 350  E) none of these",
 'result': 'A',
 'llm_answer': 'B) rs . 300',
 'timestamp': 1757263849.8436508,
 'processing_time_seconds': 0.14180803298950195}

In [15]:
num_correct = 0
total = 0
for i in responses:
    try:
        if i['result'].lower() in i['llm_answer'].lower():
            num_correct += 1
        total += 1
    except:
        print(i)

print(f"Baseline accuracy: {num_correct / total}")

{'item_index': 5472, 'id': 'math_qa__5VU5HkqG7DPDVRxL', 'question': 'a is the hundreds digit of the 3 digit integer x , b is the tens digit of x , and c is the units digit of x . 4 a = 2 b = c , and a > 0 . what is the difference between the two greatest possible values of x ? tip : dont stop till you have exhausted all answer choices to arrive at the correct one .\tAnswers.\nA) 124\nB) 297\nC) 394\nD) 421\nE) 842', 'result': 'A', 'llm_answer': None, 'timestamp': 1757264720.0093277, 'processing_time_seconds': 30.031142950057983}
Baseline accuracy: 0.34834906790626347
